<a href="https://colab.research.google.com/github/Mogaveera/atmanomalydetection/blob/master/anomaly_4_classes_using_imageclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 1.x

import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix

SEQ_LEN = 30
BATCH_SIZE = 16
EPOCHS = 100

train_video_index = []
test_video_index = []

Using TensorFlow backend.


In [0]:
!git clone https://github.com/Mogaveera/anomalydetection.git

Cloning into 'anomalydetection'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 250 (delta 0), reused 5 (delta 0), pack-reused 243
Receiving objects: 100% (250/250), 733.86 MiB | 38.45 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Checking out files: 100% (233/233), done.


In [0]:
def main():
    # Get model with pretrained weights.
    # base_model = InceptionV3(
    # weights='imagenet',
    # include_top=True)
    
    
    # We'll extract features at the final pool layer.
    # model = Model(
    #     inputs=base_model.input,
    #     outputs=base_model.get_layer('avg_pool').output)
    # # model.save('inception.h5')

    base_model = load_model('./drive/My Drive/imageclassifier3_1.h5')

    layer_name = 'flatten_1'
    model = Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)

    # Getting the data
    df = get_data('anomalydetection/Data/data_file.csv')
    
    # Clean the data
    # df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df)
    
    # Split the dataset into train and test
    train, test = split_train_test(df)
    
    # Encoding the dataset
    train_video_index = make_dataset(train, model,label_index, "train")
    test_video_index = make_dataset(test, model,label_index,"test")
    return (train_video_index, test_video_index)

In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
#def clean_data(df):
#    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
#    df = df[mask]`
#    return df

def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def video_to_frame(row, model,label_index, phase, not_created):
    input_f = []
    output_y = []
    index = 0
    cap = cv2.VideoCapture(os.path.join("anomalydetection/Data","anomaly_dataset",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".mp4")) 
    #print(str(row["class"].iloc[0]))
    #print(str(row["video_name"].iloc[0]))
    frameno = 1
    imgs = []
    length = 0
    seq = 12
    while (cap.isOpened()):
      ret, frame = cap.read()
      if ret:
        if length < seq:
          if frameno % 10 == 0:
            frameno = frameno + 1
            frame = preprocess_image(frame)
            frame = image.img_to_array(frame)
            frame = frame / 255
            # features = model.predict(frame)
            imgs.append(frame)
            length = length + 1
          else:
            frameno = frameno + 1
        else:
          seq = seq + 12
          imgs1 = np.array(imgs)
          features = model.predict(imgs1)
          input_f.append(features)
          output_y.append(label_index)
          del imgs[:]
      else:
        break

    if not_created:
      f = h5py.File(phase+'_4'+'.h5', 'w')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        f.create_dataset(phase, data=input_f1, maxshape=(None, 12, 2048))
        f.create_dataset(phase+"_labels", data=output_y1, maxshape=(None,))
        f.close()
    else:
      hf = h5py.File(phase+'_4'+'.h5', 'a')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        hf[phase].resize((hf[phase].shape[0] + input_f1.shape[0]), axis = 0)
        hf[phase][-input_f1.shape[0]:] = input_f1

        hf[phase+"_labels"].resize((hf[phase+"_labels"].shape[0] + output_y1.shape[0]), axis = 0)
        hf[phase+"_labels"][-output_y1.shape[0]:] = output_y1
        hf.close()

    del input_f[:]
    del output_y[:]
    del imgs[:]
    cap.release()
    return index
    
    


def make_dataset(data, model,label_index, phase):
    video_index = [0]
    required_classes = ["Arson","Burglary", "Fighting", "normal"]
   
    not_created = True
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
            index = required_classes.index(str(data.iloc[[i]]["class"].iloc[0]))
            # label_index = np.zeros((4))
            # label_index[index] = 1
            label_index = index
            index = video_to_frame(data.iloc[[i]], model,label_index, phase, not_created)
            real_index = video_index[-1] + index
            video_index.append(real_index)
            if real_index > 0:
              not_created = False

    return video_index

In [0]:
train_video_index, test_video_index = main()
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




100%|██████████| 39/39 [00:18<00:00,  1.15s/it]


In [0]:
x_train = HDF5Matrix('train_4.h5', 'train')
y_train = HDF5Matrix('train_4.h5', 'train_labels')
x_test = HDF5Matrix('test_4.h5', 'test')
y_test = HDF5Matrix('test_4.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[783])
print(x_test.shape)
print(y_test.shape)
print(y_test[93])
print(train_video_index)
print(test_video_index)

(786, 12, 2048)
(786,)
3
(225, 12, 2048)
(225,)
3
[0, 0, 3, 6, 10, 13, 16, 20, 21, 23, 26, 29, 32, 37, 42, 46, 49, 57, 61, 64, 68, 68, 73, 78, 83, 86, 90, 94, 96, 99, 101, 104, 105, 106, 107, 112, 114, 118, 123, 124, 128, 131, 132, 134, 136, 145, 150, 156, 160, 164, 167, 171, 173, 176, 177, 180, 197, 198, 208, 211, 222, 226, 241, 247, 248, 259, 277, 283, 292, 295, 330, 348, 374, 390, 423, 428, 435, 436, 453, 457, 477, 482, 485, 504, 513, 519, 522, 530, 542, 545, 554, 577, 585, 589, 598, 611, 621, 631, 637, 650, 658, 677, 689, 704, 706, 714, 722, 727, 730, 737, 751, 753, 759, 775, 780, 786]
[0, 1, 3, 4, 5, 9, 12, 17, 20, 26, 33, 38, 45, 52, 64, 69, 87, 93, 108, 165, 206, 215, 218, 225]


In [0]:
y_test1 = np.array(y_test)
print(y_test1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3]


In [0]:
def lstm():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (12, 2048)
    # Model.
    model = Sequential()
    model.add(Bidirectional(LSTM(2048), input_shape=input_shape))
    model.add(Dropout(0.8))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(4, activation='softmax'))
    #model.add(Dense(10, activation='softmax'))"""
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    callback_list = [checkpoint]
    
    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    #return model, callback_list
    #model.compile(optimizer = tf.train.AdamOptimizer(),
    #          loss = 'categorical_crossentropy',
    #        metrics=['accuracy'])
    return model


In [0]:
lstm = lstm()
lstm.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 4096)              67125248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
Total params: 69,224,964
Trainable params: 69,224,964
Non-trainable params: 0
___________________________________________________________

In [0]:
lstm.fit(x_train, y_train, batch_size = 16, epochs = 25,verbose = 2, validation_data = (x_test, y_test), shuffle = 'batch')
lstm.save("Anomaly_Recognition_4classes.h5")

Train on 786 samples, validate on 225 samples
Epoch 1/25
 - 10s - loss: 2.4261 - acc: 0.3104 - val_loss: 1.2134 - val_acc: 0.3333
Epoch 2/25
 - 5s - loss: 1.9737 - acc: 0.3702 - val_loss: 1.2494 - val_acc: 0.2667
Epoch 3/25
 - 5s - loss: 1.6700 - acc: 0.4338 - val_loss: 1.0971 - val_acc: 0.4756
Epoch 4/25
 - 5s - loss: 1.5592 - acc: 0.4580 - val_loss: 1.1776 - val_acc: 0.2756
Epoch 5/25
 - 5s - loss: 1.4597 - acc: 0.4389 - val_loss: 1.1485 - val_acc: 0.2978
Epoch 6/25
 - 5s - loss: 1.3596 - acc: 0.4936 - val_loss: 0.9880 - val_acc: 0.5333
Epoch 7/25
 - 5s - loss: 1.2557 - acc: 0.5153 - val_loss: 1.0302 - val_acc: 0.5244
Epoch 8/25
 - 5s - loss: 1.2088 - acc: 0.5394 - val_loss: 1.0782 - val_acc: 0.4044
Epoch 9/25
 - 5s - loss: 1.2596 - acc: 0.5293 - val_loss: 0.9629 - val_acc: 0.6178
Epoch 10/25
 - 5s - loss: 1.1599 - acc: 0.5420 - val_loss: 1.0526 - val_acc: 0.3956
Epoch 11/25
 - 5s - loss: 1.0814 - acc: 0.5865 - val_loss: 1.0111 - val_acc: 0.4978
Epoch 12/25
 - 5s - loss: 1.1168 - acc

In [0]:
test_loss, test_acc = lstm.evaluate(x_test, y_test, batch_size=16)
print("accuracy: ", test_acc)

225/225 [==============================] - 0s 2ms/step
accuracy:  0.7155555555555555
